# Verification problem

Verification of transient solution of 1D rod from the [NAFEMS](https://www.nafems.org) test suite, see section "1D Single Equation" on [wolfram.com](https://reference.wolfram.com/language/PDEModels/tutorial/HeatTransfer/HeatTransferVerificationTests.html).

In [20]:
using MMJMesh
using MMJMesh.Plots
using MMJMesh.Meshes
using MMJMesh.Utilities
using MMJMesh.Geometries

import Plots
import CairoMakie
using ProgressLogging
using DifferentialEquations

include("fem.jl")
include("heat.jl")
CairoMakie.update_theme!(colormap=:acton)

## Mesh

In [ ]:
m = Mesh("gmsh/thermal-equilibrium.msh")

nl = [i for i = nodeindices(m) if coordinate(node(m, i), 1) < 0.1]
nr = [i for i = nodeindices(m) if coordinate(node(m, i), 1) > 0.1]

p = mplot(m, edgesvisible=true) |> mconf()
CairoMakie.scatter!(coordinates(m, nl), color=:hotpink)
CairoMakie.scatter!(coordinates(m, nr), color=:steelblue)
p


## Element functions

In [4]:
setdata!(group(m, :elements), :ke_func, heat_ke(401))
setdata!(group(m, :elements), :me_func, heat_me(8960, 3850))

## Initial condition

In [ ]:
t0 = zeros(nnodes(m))
t0[nl] .= -10
t0[nr] .= 10

mplot(m, t0) |> mconf()

### Solve initial value problem

#### ODE function

In [7]:
K, M, r = assemble_kmr(m)
F(ΘHat, _, t) = -K * ΘHat
JF(_, _, _) = -K
FM = ODEFunction(F, jac=JF, mass_matrix=M);

#### Solve initial value problem

In [ ]:
luM = lu(M)
F(ΘHat, _, t) =  luM \ (-K * ΘHat)

p = ODEProblem(F, t0, (0.0, 3600.0))
@time s2 = solve(p, Euler(), dt=0.1)
Plots.plot(s2, legend=false)

In [ ]:
p = ODEProblem(F, t0, (0.0, 3600.0))
@time s = solve(p, Trapezoid(), progress=true)
Plots.plot(s, legend=false)

## Plot results

In [ ]:
mplot(m, s.u[end]) |> mconf()